### Pose Estimation
This is a small section which will help you to create some cool 3D effects with calib module.

#### Goal
In this section,
* We will learn to exploit calib3d module to create some 3D effects in images.

#### Basics
이것은 작은 부분이 될 것입니다. 카메라 보정에 대한 마지막 세션에서 카메라 매트릭스, 왜곡 계수 등을 찾았습니다. 패턴 이미지가 주어지면 위의 정보를 활용하여 포즈를 계산하거나 회전하는 방법과 같이 공간에 객체를 배치하는 방법을 사용할 수 있습니다. 평면 객체의 경우 Z = 0이라고 가정하면 문제는 카메라가 공간에 배치되어 패턴 이미지를 볼 수 있게 됩니다. 따라서 우리가 물체가 어떻게 공간에 있는지 알게되면 3D 효과를 시뮬레이트하기 위해 2D 다이어그램을 그려 볼 수 있습니다. 그것을 하는 방법을 보자.

우리의 문제는 체스 판의 첫 번째 구석에 3D 좌표축 (X, Y, Z 축)을 그려야한다는 것입니다. X 축은 파란색, Y 축은 녹색 및 Z 축은 빨간색으로 표시됩니다. 결과적으로 Z 축은 체스 판 평면에 수직 인 것처럼 느껴져야합니다.

먼저, 이전 보정 결과에서 카메라 행렬 및 왜곡 계수를로드합니다.

In [3]:
import numpy as np
import cv2 as cv
import glob

# Load previously saved data
with np.load('B.npz') as X:
    mtx, dist, _, _ = [X[i] for i in ('mtx','dist','rvecs','tvecs')]

Now let's create a function, draw which takes the corners in the chessboard (obtained using [cv.findChessboardCorners()](https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#ga93efa9b0aa890de240ca32b11253dd4a)) and **axis points** to draw a 3D axis.

In [4]:
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv.line(img, corner, tuple(imgpts[0].ravel()), (255, 0, 0), 5)
    img = cv.line(img, corner, tuple(imgpts[1].ravel()), (0, 255, 0), 5)
    img = cv.line(img, corner, tuple(imgpts[2].ravel()), (0, 0, 255), 5)
    return img

그런 다음 앞의 경우처럼 종결 기준, 객체 점 (체스 판의 모서리 3D 점) 및 축 점을 만듭니다. 축 점은 축을 그리기위한 3D 공간의 점입니다. 우리는 길이 3의 축을 그립니다 (단위는 그 크기에 따라 조정 한 이후로 체스의 정사각형 크기를 기준으로합니다). 따라서 우리의 X 축은 (0,0,0)에서 (3,0,0)까지 그려 지므로 Y 축에 대해서도 마찬가지입니다. Z 축의 경우 (0,0,0)에서 (0,0, -3)까지 그려집니다. 음수는 카메라쪽으로 그려 짐을 나타냅니다.

In [5]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6 * 7, 3), np.float32)
objp[:,:2] = np.mgrid[0:7, 0:6].T.reshape(-1, 2)
axis = np.float32([[3, 0, 0], [0, 3, 0], [0, 0, -3]]).reshape(-1,3)

이제 평상시처럼 각 이미지를로드합니다. 7x6 그리드 검색. 발견되면 하위 픽셀로 수정합니다. 그런 다음 회전 및 평행 이동을 계산하기 위해 [cv.solvePnPRansac()](https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#ga50620f0e26e02caa2e9adc07b5fbf24e) 함수를 사용합니다 . 일단 우리가 이러한 변환 행렬을 사용하면 우리는 그것들을 사용하여 축 포인트 를 이미지 평면에 투영합니다. 간단히 말하면, 3D 공간에서 (3,0,0), (0,3,0), (0,0,3) 각각에 해당하는 이미지 평면상의 점을 찾습니다. 일단 우리가 얻으면 우리는 draw () 함수를 사용하여 첫 번째 모서리에서부터이 점들까지 선들을 그립니다. 완료되었습니다!

In [11]:
for fname in glob.glob('./07CalibrationData/*.jpg'):
    img = cv.imread(fname)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, (7, 6), None)
    if ret == True:
        corners2 = cv.cornerSubPix(gray, corners,(11, 11),(-1, -1), criteria)
        
        # Find the rotation and translation vectors.
        ret,rvecs, tvecs = cv.solvePnP(objp, corners2, mtx, dist)
        
        # project 3D points to image plane
        imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, mtx, dist)
        img = draw(img, corners2, imgpts)
        cv.imshow('img',img)
        k = cv.waitKey(0) & 0xFF
        if k == ord('s'):
            cv.imwrite(fname[20:26] + '.png', img)
cv.destroyAllWindows()
cv.waitKey(1)

-1

##### Render a Cube
If you want to draw a cube, modify the draw() function and axis points as follows.

Modified draw() function:

In [9]:
def draw(img, corners, imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)
    # draw ground floor in green
    img = cv.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
    # draw top layer in red color
    img = cv.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img

Modified axis points. They are the 8 corners of a cube in 3D space:

In [10]:
axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
                   [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ])

If you are interested in graphics, augmented reality etc, you can use OpenGL to render more complicated figures.

#### Additional Resources

#### Exercises